# SciPy, Pingouin, SciKit-PostHocs

**SciPy** (Scientific Python) – biblioteka do różnego typu obliczeń matematycznych w nauce i inżynierii
- Jej podmoduł **stats** zawiera szeroki zbiór funkcji statystycznych. 
- Na tej bibliotece oraz NumPy zbudowana jest biblioteka scikit-learn służąca do uczenia maszynowego.

**Pingouin** - biblioteka do operacji statystycznych, z przyjazną składnią i rozbudowanymi outputami dla najczęściej używanych testów w różnych dziedzinach nauki, m.in. naukach społecznych

**SciKit-PostHocs** (SciPy Toolkit - Post-hoc Tests) - biblioteka uzupełniająca SciPy.Stats, zawierająca testy post-hoc

**Statsmodels** (Statistical Models) - biblioteka uzupełniająca SciPy, zawierająca modele statystyczne

---

Instalacja bibliotek:

In [ ]:
%pip install scipy
%pip install pingouin
%pip install scikit-posthocs
%pip install statsmodels

Załadowanie bibliotek:

In [ ]:
import scipy.stats as st
import pingouin as pg
import scikit_posthocs as sp
import statsmodels.formula.api as smf

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np   

plt.rcParams['figure.figsize'] = (4, 3)  

import warnings

warnings.simplefilter(category = FutureWarning, action = 'ignore')

Wczytanie danych:

In [ ]:
data = pd.read_csv('psych_health_adaptive_learning_data.csv')

# wygenerowanie dwóch kolumn na potrzeby materiałów

np.random.seed(10) 

data.insert(column = 'attention_score',
            value  = np.where(data['gender'] == 'female',
                              np.random.normal(0.6, 0.15, 300).round(2),
                              np.random.normal(0.65, 0.15, 300).round(2)) - 0.05,
            loc    = data.columns.get_loc('engagement_score') + 1)
data.insert(column = 'mood_variability_score',
            value  = np.where(data['topic'] == 'Mindfulness',
                              np.round(np.random.normal(0.5, 0.15, 300), 2),
                     np.where(data['topic'] == 'Coping skills',         
                              np.round(np.random.normal(0.55, 0.15, 300), 2),
                     np.where(data['topic'] == 'Stress management',         
                              np.round(np.random.normal(0.45, 0.15, 300), 2),      
                              np.round(np.random.normal(0.55, 0.15, 300), 2)))),
            loc    = data.columns.get_loc('attention_score') + 1)

Czyszczenie danych:

In [ ]:
# usunięcie kolumny z jedną wartością

data = data.drop(columns = 'country')

# poprawa poszczególnych zmiennych

data = data.apply(lambda col: col.str.lower() if col.dtype == 'object' else col
          ).assign(gender = data['gender'].map({'F':'female', 'M':'male'}))

# poprawa typów zmiennych

data = data.astype({'user_id'        : 'str',
                    'content_id'     : 'str',
                    'timestamp'      : 'datetime64[ns]'}
          ).assign(education_level = data['education_level'].astype('category').cat.reorder_categories(
                                           ['diploma', 'undergrad', 'postgrad', 'phd'], ordered = True),
                   difficulty      = data['difficulty'].astype('category').cat.reorder_categories(
                                           ['easy', 'medium', 'hard'], ordered = True),
                   feedback_rating = data['feedback_rating'].astype('category').cat.reorder_categories(
                                           [1, 2, 3, 4, 5], ordered = True),
                   reward_category = data['reward_category'].astype('category').cat.reorder_categories(
                                           ['low', 'medium', 'high'], ordered = True))

Dane dotyczą interakcji użytkowników z treściami edukacyjnymi z zakresu zdrowia psychicznego na platformie cyfrowej:

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
# wybranie podzbioru użytkowników

users_data = data.filter(['user_id', 'age', 'gender', 'education_level', 'occupation', 'baseline_stress_score', 'baseline_resilience_score']
                ).drop_duplicates(ignore_index = True).drop(columns = 'user_id')
users_data.info()

# 1. Statystyczne testowanie hipotez

Rozkład zmiennej losowej opisuje prawdopodobieństwo wylosowania poszczególnych jej wartości.

**Przykład 1**:

> $X$ - wynik rzutu kostką
> 
> Rozkład $X$:
> 
> $P(X = 1)$    | $P(X = 2)$    | $P(X = 3)$    | $P(X = 4)$    | $P(X = 5)$     | $P(X = 6)$     
> --------------|---------------|---------------|---------------|---------------|---------------
> $\frac{1}{6}$ | $\frac{1}{6}$ | $\frac{1}{6}$ | $\frac{1}{6}$ | $\frac{1}{6}$ | $\frac{1}{6}$

**Przykład 2**:

> $X$ - liczba wyrzuconych reszek przy potrójnym rzucie monetą
> 
> Rozkład $X$:
>
> $P(X = 0)$    | $P(X = 1)$    | $P(X = 2)$    | $P(X = 3)$    
> --------------|---------------|---------------|---------------
> $\frac{1}{8}$ | $\frac{3}{8}$ | $\frac{3}{8}$ | $\frac{1}{8}$ 

 **Przykład 3**:

> $X \sim N(0, 1)$
>
> Rozkład $X$:

In [ ]:
x = np.linspace(-4, 4, 400); y = st.norm.pdf(x, 0, 1)

plt.figure(figsize = (3, 2))
plt.plot(x, y)
plt.xlabel('\n$x$')
plt.ylabel('gęstość\n')
plt.show()

---

## Idea testów statystycznych

Każda kolumna w danych (w postaci tidy), to pewien zbiór wartości wylosowany z rozkładu odpowiedniej **zmiennej losowej**, który może nie być nam znany. **Celem** statystycznego testowania hipotez jest **rozpoznanie właściwości tego rozkładu**. Jeśli właściwością, którą chcemy zbadać jest pewien **parametr rozkładu** jak średnia lub wariancja, to musimy skorzystać z jednej z metod parametrycznych, a w przeciwnym razie, z jednej z nieparametrycznych. 

<!-- Każda cecha ze względu na którą można podzielić poszczególnych osobników populacji, to pewna zmienna losowa, której pewien zbiór wartości losujemy przy tworzeniu próby z populacji. Losowanie to odbywa się więc z rozkładu tworzonego przez odsetki odpowiednich podgrup populacji tworzonych przez tę cechę. Cechy psychologiczne w ludzkiej populacji mają najczęściej rozkład zbliżony do rozkładu normalnego o pewnych niemożliwych dla nas do poznania parametrach jakimi są średnia i odchylenie standardowe. -->

## Idea testów statystycznych w krokach

### 1. Hipoteza zerowa

Ustalenie $\theta_0$ - wartości parametru $\theta$, która zostanie rozważona. \
Wtedy zdanie $\theta = \theta_0$ będzie naszą tzw. hipotezą zerową - naszym założeniem, i oznaczymy ją poprzez $H_0$.

> *Przykład:* 
>   
>   hipoteza   - "średni wzrost mieszkańców Poznania to 1.67 m"
>  
>   $X$    - wzrost na populacji "mieszkańcy Poznania" [m] \
>   $\mu$  - średni wzrost mieszkańców Poznania \
>   $1.67$ - wybrany wzrost mieszkańców Poznania, który chcemy rozważyć
>   
>   $H_0: \mu = 1.67$

### 2. Hipoteza alternatywna

Określenie hipotezy alternatywnej $H_1$, którą rozważymy w parze z $H_0$: 

- $H_1: \theta \neq \theta_0$
- $H_1: \theta > \theta_0$
- $H_1: \theta < \theta_0$ 

> *Przykład:* 
>   
> $H_1: \mu \neq 1.67$

### 3. Statystyka z próby
  
Odpowiedź na postawioną hipotezę możemy znaleźć, analizując próbę z wybranej populacji. \
Obliczenie estymatora parametru $\theta$ z próby - odpowiedniej statystyki, np. średniej (empirycznej). 

> *Przykład:* 
>   
> $x$    - próba - kolumna w danych zebranych na części mieszkańców Poznania dotycząca wzrostu \
> $1.74$ - średni wzrost w próbie
>
>
>
>   
>                                   ~~ ~~
>                                ~~   |   ~~
>                             ~~      |      ~~
>                          ~~         |        |~~
>                       ~~            |        |    ~~
>                    ~~               |        |       ~~
>        _______ ~~ __________________|________|__________ ~~ _______
>                               m0 = 1.67  m = 1.74 
>
>
>
> - średnia empiryczna przy założonej wartości parametru populacji całkiem prawdopodobna $\rightarrow$ może i $H_0$
> - średnia empiryczna przy założonej wartości parametru populacji mało prawdopodobna $\rightarrow$ chyba nie $H_0$

### 4. Statystyka testowa

W określeniu prawdopodobieństwa otrzymanego wyniku pomagają nam stworzone **rozkłady teoretyczne**, jak rozkład normalny, Studenta czy Fishera, z których każdy ma opisywać jakieś zjawisko w świecie i stanowić wzorzec dla oczekiwanych wartości zmiennych z danej klasy.

Na przykład:

 - rzut kostką powinien mieć rozkład jednostajny,
 - liczba orłów w kilku rzutach monetą rozkład dwumianowy,
 - cechy psychologiczne w ludzkiej populacji założono, że powinny mieć rozkład zbliżony do rozkładu normalnego.    

Musimy więc założyć, z którego rozkładu pochodzi nasza zmienna i tak przekształcić naszą statystykę, aby móc skorzystać z jednego z "gotowych" rozkładów teoretyczych. Przekształcenie to nazywa się **statystyką testową** i jest, tak samo jak statystyka średniej, pewną funkcją próby. Wzory według których warto w tym celu przekształcić statystykę w zależności od wiedzy o badanej zmiennej, zostały ogólnie przyjęte jako **testy statystyczne** o określonej nazwie, często od twórcy danego wzoru.

>   *Przykład:*
>
> - $X \sim N(\mu, \sigma^2)$
> - $\overline{X} \sim N(\mu, \frac{\sigma^2}{n})$
>
> $$ Z(X) = \frac{m - \mu_0}{\sqrt{\frac{\sigma^2}{n}}} \sim N(0, 1) $$
>
> $$ T(X) = \frac{m - \mu_0}{\sqrt{\frac{s^2}{n}}} \sim t_{\mbox{ }n - 1} $$
>
> Załóżmy, że w naszym dotychczasowym przykładzie statystyka odchylenia standardowego wyniosła 0.15, a próba liczyła 30 osób. Wtedy:
>
> $$ T = \frac{1.74 - 1.67}{\sqrt{\frac{0.15^2}{30}}} = 2.56 $$
>
>
>
>   
>                                   ~~ ~~
>                                ~~   |   ~~
>                             ~~      |      ~~
>                          ~~         |        |~~
>                       ~~            |        |    ~~
>                    ~~               |        |       ~~
>        _______ ~~ __________________|________|__________ ~~ _______
>                                     0    T = 2.56 
>
>

### 5. Wartość $p$

Statystyka testowa może być podstawą do odrzucenia $H_0$ na rzecz $H_1$, jeżeli jest zbyt mało prawdopodobne, aby zaobserwowana jej wielkość mogła wystąpić przez przypadek przy założeniu prawdziwości $H_0$. "mało prawdopodobne" oznacza, że prawdopodobieństwo, które potrzebujemy obliczyć, to prawdopodobieństwo otrzymania wartości $T$ lub bardziej skrajnej. Wartość tę nazywamy **wartością $p$**.

Przy poszczególnych hipotezach alternatywnych wartość $p$ będzie odpowiadać:

- $H_1: \theta \neq \theta_0$ - prawdopodobieństwu lewego i prawego ogona rozkładu
- $H_1: \theta > \theta_0$    - prawdopodobieństwu prawego ogona rozkładu
- $H_1: \theta < \theta_0$    - prawdopodobieństwu lewego ogona rozkładu

> *Przykład:*
>
> $$ p = 2 \cdot (1 - t_{n - 1}(|T|)) = 0.0003 $$

In [ ]:
from scipy.special import stdtr

m   = 1.74
mu0 = 1.67
s   = 0.15
n   = 30

T = (m - mu0) / np.sqrt(s**2 / n)
p = 2 * (1 - stdtr(abs(T), n - 1))

(round(float(T), 5), round(float(p), 5))

### 6. Wynik testu statystycznego

Aby podjąć decyzję o odrzuceniu $H_0$ musimy określić, czy $p$ jest do tego wystarczająco małe. W tym pomaga nam tzw. **poziom istotności $\alpha$**, który powinniśmy ustalić jeszcze przed wykonaniem testu statystycznego (nie po zobaczeniu wartości $p$). Jest on górną granicą dla $p$, po której przekroczeniu uznamy, że **nie mamy podstaw do odrzucenia $H_0$** i kwestia wartości parametru pozostaje dla nas bez zmian. Jeśli jednak $p$ okaże się mniejsze lub równe $\alpha$, zdecydujemy się odrzucić $H_0$, co sprawi, że zbiór potencjalnych wartości szukanego parametru zmniejszy się o 1.

Najczęstszym poziomem ufności jest 0.05, ale można spotkać również $\alpha = 0.1$, $\alpha = 0.01$ lub $\alpha = 0.001$. 

> *Przykład:*
> 
> Niech $\alpha = 0.05$.
> 
> $$ p = 0.0003 < 0.05 = \alpha $$
>
> Odrzucamy $H_0$ mówiącą, że $\mu = 1.67$, a więc średni wzrost mieszkańców Poznania nie wynosi 1.67 ( $T(29) = 2.56$; $p < 0.001$ ).


### 7. Wielkość efektu

Jeśli w wyniku testu statystycznego odrzucamy hipotezę zerową, warto uzupełnić go o **wielkość efektu**, który stwierdzono, czyli ilościową miarę siły zjawiska, wyrażoną liczbą z przedziału [0, 1]. Podobnie jak w przypadku statystyki testowej, oblicza się ją wg ustalonego wzoru na podstawie danych. Między innymi silnie zależy ona od wielkości próby, więc poza celami informacyjnymi, może służyć do zaplanowania liczby osób, które należy zbadać, aby uzyskać w badaniu określoną wielkość efektu. Wykorzystuje się ją też do nadania wag poszczególnym wynikom w metaanalizach podsumowujących badania z danego obszaru nauki.

Najpopularniejsza miara wielkości efektu to $d$ Cohena, dla którego jej twórca zdefiniował następującą interpretację:

- $|d| < 0.5$ &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;         - efekt mały,
- $0.5 \leqslant |d| < 0.8$                                                        - efekt średni,
- $0.8 \leqslant |d|$ &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; - efekt duży.

> *Przykład:*
> 
> $$ d = \frac{m - \mu_0}{s} = \frac{1.74 - 1.67}{0.15} = 0.47 $$
>
> A zatem ostatecznie:
> 
> Odrzucamy $H_0$ mówiącą, że $\mu = 1.67$, a więc średni wzrost mieszkańców Poznania nie wynosi 1.67 ( $T(29) = 2.56$; $p < 0.001$; $d = 0.47$ ). Efekt ten jest mały.



---

Powyższy przykład testu statystycznego to test Studenta jednej próby.

## Test Studenta jednej próby

Założenia:

- $X \sim N(\mu, \sigma^2)$ &nbsp;&nbsp;&nbsp; ($X$ ma rozkład normalny o nieznanych parametrach)

### Przykład

zmienna ($X$) - wskaźnik uwagi (attention_score)

Hipoteza badawcza: Średni wskaźnik uwagi w populacji nie wynosi 0.58.

$H_0: \mu = 0.58$ \
$H_1: \mu \neq 0.58$  

$\alpha = 0.05$  &nbsp;&nbsp;&nbsp;&nbsp;   (będziemy tak przyjmować też w pozostałych testach)

**Rozkład zmiennej w próbie:**

In [ ]:
plt.figure(figsize = (3, 2))

plt.hist(data['attention_score'],                            
         color     = '#34c3e9',
         edgecolor = 'white')
plt.xlabel('wskaźnik uwagi', size = 10)             
plt.ylabel('częstość', size = 10)                                                                                                                                      
plt.show()

data['attention_score'].agg(['mean', 'std'])

**Test wstępny**, badający normalność zmiennej - test Shapiro-Wilka:

Hipoteza badawcza: Wskaźnik uwagi w populacji ma rozkład normalny.

$H_0:$ Wskaźnik uwagi w populacji ma rozkład normalny. \
$H_1:$ Wskaźnik uwagi w populacji nie ma rozkładu normalnego.

In [ ]:
# biblioteka scipy

st.shapiro(data['attention_score'])

In [ ]:
# biblioteka pingouin

pg.normality(data['attention_score'])

> *Wniosek:* \
> Na poziomie istotności 0.05 nie mamy podstaw do odrzucenia hipotezy zerowej ( $W = 0.99$ ; $p = 0.77 $ ), a więc przyjmiemy, że wskaźnik uwagi w populacji wystarczająco spełnia założenie normalności testu Studenta.

W przypadku odrzucenia $H_0$ użylibyśmy testu Wilcoxona dla jednej próby.

**Test docelowy:**

In [ ]:
# biblioteka scipy

st.ttest_1samp(data['attention_score'], 0.58)

In [ ]:
# biblioteka pingouin

pg.ttest(data['attention_score'], 0.58)

> *Wniosek:* \
> Wskaźnik uwagi w populacji nie wynosi 0.58 ( $m = 0.60$ ; $sd = 0.14$ ) - efekt ten jest słaby ( $T(299) = 2.25$ ; $p = 0.03$ ; $d = 0.13$ ).

---

## Zbiór testów statystycznych

Testy statystyczne mogą badać właściwości rozkładu jednej zmiennej lub rozkładu łącznego wielu zmiennych.

Wybór odpowiedniego do problemu testu może zależeć od:
- liczby zmiennych,
- typu zmiennych,
- pochodzenia wartości zmiennych od tych samych obiektów,
- liczby poziomów zmiennych, pojedynczo i łącznie,
- spełniania przez zmienne specyficznych założeń poszczególnych testów (np. normalność rozkładu),
- specyfiki posiadanych danych oraz właściwości poszczególnych testów - warunków, w których mają najwyższą moc statystyczną.

Testy dotyczące rozkładów dwóch zmiennych można podzielić na:
- **testy istotności różnic** zmiennej nienominalnej względem grup wydzielonych przez zmienną nominalną,
- **testy siły związku** między dwoma zmiennymi nienominalnymi lub dwoma zmiennymi nominalnymi.
 
Pomimo użytych określeń, oba typy testów, również pierwszy, opisują de facto siłę związku pomiędzy jakimiś dwoma próbami, więc granica między tymi klasami się zaciera.

# 2. Testy istotności różnic dla danych niezależnych

## 2.1. 2 grupy

### Parametryczne

- *Zmienna ilościowa o rozkładzie normalnym w grupach i 2 grupy wyznaczone przez zmienną nominalną*

    Test Welcha (Studenta) dla 2 prób

$H_0: \mu_1 = \mu_2$ \
$H_1: \mu_1 \neq \mu_2$  

Założenia:

- Badana zmienna ma w grupach rozkład normalny.

#### Przykład:

zmienna - wskaźnik uwagi \
grupy - płeć

Hipoteza badawcza: Kobiety i mężczyźni różnią się wskaźnikiem uwagi.

$H_0:$ Średni wskaźnik uwagi u kobiet jest równy średniemu wskaźnikowi uwagi u mężczyzn. \
$H_1:$ Średni wskaźnik uwagi u kobiet jest różny od średniego wskaźnika uwagi u mężczyzn.

**Rozkład zmiennej w grupach:**

In [ ]:
data.groupby('gender')['attention_score'].hist(bins = np.linspace(0, 1, 21), density = True, alpha = 0.3, grid = False, legend = True)
data.groupby('gender')['attention_score'].describe()

**Test wstępny** - normalność zmiennej - test Shapiro-Wilka:

Hipoteza badawcza: Wskaźnik uwagi wśród grup poszczególnych płci ma rozkład normalny.

$H_0:$ Średni wskaźnik uwagi wśród grup poszczególnych płci ma rozkład normalny. \
$H_1:$ Średni wskaźnik uwagi wśród grup poszczególnych płci nie ma rozkładu normalnego.

In [ ]:
pg.normality(smf.ols('attention_score ~ C(gender)', data = data).fit().resid)

> *Wniosek:* \
> Na poziomie istotności 0.05 nie mamy podstaw do odrzucenia hipotezy zerowej ( $W = 1.00$ ; $p = 0.89$ ), a więc przyjmiemy, że wskaźnik uwagi wśród grup poszczególnych płci wystarczająco spełnia założenie normalności testu Studenta.

**Test docelowy** - test Welcha (Studenta) dla 2 prób niezależnych:

In [ ]:
# utworzenie podserii zmiennej ilościowej

attention_female = data.loc[data['gender'] == 'female', 'attention_score']
attention_male   = data.loc[data['gender'] == 'male',   'attention_score']

In [ ]:
# scipy

st.ttest_ind(attention_female, attention_male, equal_var = False)  

In [ ]:
# pingouin

pg.ttest(attention_female, attention_male, correction = True)  

> *Wniosek:* \
> Na poziomie istotności 0.05 nie możemy stwierdzić, czy wskaźnik uwagi u kobiet ( $m = 0.59$ ; $sd = 0.14$ ) różni się od wskaźnika uwagi u mężczyzn ( $m = 0.59$ ; $sd = 0.14$ ) ( $T(239.17) = -1.03$ ; $p = 0.30$ ).

**Test docelowy** - jednostronny test Welcha (Studenta) dla 2 prób niezależnych:

Hipoteza badawcza: Kobiety mają wyższy wskaźnik uwagi niż mężczyźni.

$H_1:$ Średni wskaźnik uwagi u kobiet jest większy od średniego wskaźnika uwagi u mężczyzn.

In [ ]:
# scipy

st.ttest_ind(attention_female, attention_male, equal_var = False, alternative = 'greater')  

In [ ]:
# pingouin

pg.ttest(attention_female, attention_male, correction = True, alternative = 'greater')

**Test docelowy** - jednostronny test Welcha (Studenta) dla 2 prób niezależnych:

Hipoteza badawcza: Kobiety mają niższy wskaźnik uwagi niż mężczyźni.

$H_1:$ Średni wskaźnik uwagi u kobiet jest mniejszy od średniego wskaźnika uwagi u mężczyzn.

In [ ]:
# scipy

st.ttest_ind(attention_female, attention_male, equal_var = False, alternative = 'less') 

In [ ]:
# pingouin

pg.ttest(attention_female, attention_male, correction = True, alternative = 'less')

### Nieparametryczne

- *Zmienna ilościowa o rozkładzie w grupach różnym od normalnego i 2 grupy wyznaczone przez zmienną nominalną*
- *Zmienna porządkowa i 2 grupy wyznaczone przez zmienną nominalną*

    Test Manna Whitneya

$H_0: F_1 = F_2$ \
$H_1: F_1 \neq F_2$  

#### Przykład:

zmienna - poziom wykształcenia \
grupy - płeć

Hipoteza badawcza: Kobiety i mężczyźni różnią się poziomem wykształcenia.

$H_0:$ Mediana poziomu wykształcenia u kobiet jest równa medianie poziomu wykształcenia u mężczyzn. \
$H_1:$ Mediana poziomu wykształcenia u kobiet jest różna od mediany poziomu wykształcenia u mężczyzn.

**Rozkład zmiennej w grupach:**

In [ ]:
# dopisanie do danych zmiennej porządkowej w postaci rangowej

users_data['education_level_rank'] = users_data['education_level'].factorize(sort = True)[0] + 1

In [ ]:
# rozkład

users_data.groupby('gender')['education_level'].value_counts(sort = False).plot(kind = 'bar')
users_data.groupby('gender')['education_level_rank'].describe()

**Test docelowy** - test Manna Whitneya:

In [ ]:
# utworzenie podserii zmiennej porządkowej w postaci rangowej

education_female = users_data.loc[users_data['gender'] == 'female', 'education_level_rank']
education_male   = users_data.loc[users_data['gender'] == 'male',   'education_level_rank']

In [ ]:
# scipy

st.mannwhitneyu(education_female, education_male)

In [ ]:
# pingouin

pg.mwu(education_female, education_male)

> *Wniosek:* \
> Kobiety różnią się poziomem wykształcenia ( $med = 2$ ; $IQR = 2$ ) od mężczyzn ( $med = 3$ ; $IQR = 2$ ) ( $U = 190$ ; $p = 0.02$ ; $r_{bc} = -0.37$ ; $CL = 0.31$ ).

## 2.2. Więcej niż 2 grupy

### Parametryczne

- *Zmienna ilościowa o rozkładzie normalnym w grupach i kilka grup wyznaczonych przez zmienną nominalną*

    Test jednoczynnikowa ANOVA

$H_0: \mu_1 = \mu_2 = $ ... $ = \mu_k $ \
$H_1: \mu_i \neq \mu_j$  

Założenia:

- Badana zmienna ma w grupach rozkład normalny.
- Badana zmienna ma w grupach homogeniczną (równą) wariancję.

#### Przykład:

zmienna - wskaźnik zmienności nastroju \
grupy - temat treści

Hipoteza badawcza: Tematy treści różnią się zmiennością nastroju jaka następuje przy interakcji z nimi.

$H_0:$ Średni wskaźnik zmienności nastroju w trakcie interakcji z treściami jest taka sama dla wszystkich tematów tych treści.\
$H_1:$ Średni wskaźnik zmienności nastroju w trakcie interakcji z treściami różni się w co najmniej jednej parze tematów tych treści.

**Rozkład zmiennej w grupach:**

In [ ]:
data.groupby('topic').boxplot(column = 'mood_variability_score', layout = (1, 4), figsize = (8, 3), fontsize = 9)
data.groupby('topic')['mood_variability_score'].describe()

**Testy wstępne:**
-  normalność zmiennej w grupach - test Shapiro-Wilka:

In [ ]:
pg.normality(smf.ols('mood_variability_score ~ C(topic)', data = data).fit().resid)

> *Wniosek:* \
> Na poziomie istotności 0.05 nie mamy podstaw do odrzucenia hipotezy zerowej ( $W = 1.00$ ; $p = 0.55$ ), a więc przyjmiemy, że wskaźnik zmienności nastroju w trakcie interakcji z treściami wśród poszczególnych ich tematów wystarczająco spełnia założenie normalności w teście ANOVA.

-  homogeniczność (równość) wariancji w grupach - test Levene'a:

    Test Levene'a

$H_0: \sigma_1^2 = \sigma_2^2 = $ ... $ = \sigma^2_k$ \
$H_1: \sigma_i^2 \neq \sigma^2_j$  

#### Przykład:

zmienna - wskaźnik zmienności nastroju \
grupy - temat treści

Hipoteza badawcza: Tematy treści mają równą wariancję zmienności nastroju jaka następuje przy interakcji z nimi.

$H_0:$ Wariancja wskaźnika zmienności nastroju w trakcie interakcji z treściami jest taka sama dla wszystkich tematów tych treści.\
$H_1:$ Wariancja wskaźnika zmienności nastroju w trakcie interakcji z treściami różni się w co najmniej jednej parze tematów tych treści.

In [ ]:
pg.homoscedasticity(data, dv = 'mood_variability_score', group = 'topic')

> *Wniosek:* \
> Na poziomie istotności 0.05 nie mamy podstaw do odrzucenia hipotezy zerowej ( $W = 0.31$ ; $p = 0.82$ ), a więc przyjmiemy, że wskaźnik zmienności nastroju w trakcie interakcji z treściami wśród poszczególnych ich tematów wystarczająco spełnia założenie homogeniczności wariancji testu ANOVA.

#### $\rightarrow$ Wariancje homogeniczne

**Test docelowy** - test jednoczynnikowa ANOVA:

In [ ]:
pg.anova(data, dv = 'mood_variability_score', between = 'topic')

> *Wniosek:* \
> Tematy treści różnią się zmiennością nastroju jaka następuje przy interakcji z nimi ( $F(3, 296) = 3.59$ ; $p = 0.01$ ; $\eta^2 = 0.04$ ).

**Testy post-hoc** - testy HSD Tukeya:

    Test HSD Tukeya

$H_0: \mu_i = \mu_j $ \
$H_1: \mu_i \neq \mu_j$  

In [ ]:
pg.pairwise_tukey(data, dv = 'mood_variability_score', between = 'topic').round(4)

> *Wniosek:* \
> Temat umiejętności radzenia sobie różni się zmiennością nastroju jaka następuje przy interakcji z nimi ( $m = 0.54$ ; $sd = 0.16$ ) od tematu uważności ( $m = 0.47$ ; $sd = 0.15$ ) ( $T(296) = 2.70$ ; $p = 0.04$ ; $g = 0.41$ ).
>
> Temat uważności różni się zmiennością nastroju jaka następuje przy interakcji z nimi od tematu zarządzania stresem ( $m = 0.54$ ; $sd = 0.15$ ) ( $T(296) = -2.63$ ; $p = 0.04$ ; $g = 0.43$ ).
>
> Dla pozostałych par tematów na poziomie istotności 0.05 nie można stwierdzić istotnej różnicy w zmienności nastroju jaka następuje przy interakcji z nimi.

#### $\rightarrow$ Wariancje niehomogeniczne

Gdyby wariancje w wykonanym teście Levene'a okazały się niehomogeniczne, to należałoby wykonać modyfikację testu ANOVA, która nie posiada takowego założenia:

    Test jednoczynnikowa ANOVA Welcha

$H_0: \mu_1 = \mu_2 = $ ... $ = \mu_k $ \
$H_1: \mu_i \neq \mu_j$  

Założenia:

- Badana zmienna ma w grupach rozkład normalny.

**Test docelowy** - test jednoczynnikowa ANOVA Welcha:

In [ ]:
pg.welch_anova(data, dv = 'mood_variability_score', between = 'topic')

> *Wniosek:* \
> Tematy treści różnią się zmiennością nastroju jaka następuje przy interakcji z nimi ( $F(3, 161.72) = 3.73$ ; $p = 0.01$ ; $\eta^2 = 0.04$ ).

**Testy post-hoc** - testy Gamesa-Howella:

In [ ]:
pg.pairwise_gameshowell(data, dv = 'mood_variability_score', between = 'topic').round(4)

> *Wniosek:* \
> Temat umiejętności radzenia sobie różni się zmiennością nastroju jaka następuje przy interakcji z nimi ( $m = 0.54$ ; $sd = 0.16$ ) od tematu uważności ( $m = 0.47$ ; $sd = 0.15$ ) ( $T(195.51) = 2.66$ ; $p = 0.04$ ; $g = 0.41$ ).
>
> Temat uważności różni się zmiennością nastroju jaka następuje przy interakcji z nimi od tematu budowania odporności ( $m = 0.54$ ; $sd = 0.15$ ) ( $T(136.40) = -2.67$ ; $p = 0.04$ ; $g = 0.44$ ).
>
> Temat uważności różni się zmiennością nastroju jaka następuje przy interakcji z nimi od tematu zarządzania stresem ( $m = 0.54$ ; $sd = 0.15$ ) ( $T(147.90) = -2.67$ ; $p = 0.04$ ; $g = 0.43$ ).
>
> Dla pozostałych par tematów na poziomie istotności 0.05 nie można stwierdzić istotnej różnicy w zmienności nastroju jaka następuje przy interakcji z nimi.

### Nieparametryczne

- *Zmienna ilościowa o rozkładzie różnym w grupach od normalnego i kilka grup wyznaczonych przez zmienną nominalną*
- *Zmienna porządkowa i kilka grup wyznaczonych przez zmienną nominalną*

    Test Kruskala-Wallisa

$H_0: F_1 = F_2 = $ ... $ = F_k $ \
$H_1: F_i \neq F_j$ 

#### Przykład:

zmienna - poziom trudności \
grupy - typ treści

Hipoteza badawcza: Typy treści różnią się poziomem trudności.

$H_0:$ Mediana poziomu trudności jest taka sama dla wszystkich typów treści.\
$H_1:$ Mediana poziomu trudności różni się w co najmniej jednej parze typów treści.   

**Rozkład zmiennej w grupach:**

In [ ]:
# dopisanie do danych zmiennej porządkowej w postaci rangowej

data['difficulty_rank'] = data['difficulty'].factorize(sort = True)[0] + 1

In [ ]:
# rozkład

data.groupby('content_type')['difficulty'].value_counts(sort = False).plot(kind = 'bar')
data.groupby('content_type')['difficulty_rank'].describe()

**Test docelowy** - test Kruskala-Wallisa:

In [ ]:
pg.kruskal(data, dv = 'difficulty_rank', between = 'content_type')    # zmienna porządkowa w postaci rangowej

> *Wniosek:* \
> Typy treści różnią się poziomem trudności ( $H(3) = 25.71$ ; $p < 0.001$ ; $\eta^2 = 0.09$ ).

Wielkość efektu $\eta^2$ w powyższym wniosku to statystyka $\frac{H}{n - 1}$.

**Testy post-hoc** - testy Dunn:

    Test Dunn

$H_0: F_i = F_j $ \
$H_1: F_i \neq F_j$  

In [ ]:
# tablica wartości p dla poszczególnych par grup

difficulty_content_dunn_p = sp.posthoc_dunn(data, val_col = 'difficulty', group_col = 'content_type', p_adjust = 'fdr_bh')
difficulty_content_dunn_p.round(4)

In [ ]:
# test Dunn z parami grup w kolumnach

def dunn_p_pair_table(df_p):
    return df_p.stack(
              ).reset_index(
              ).query('level_0 < level_1'
              ).rename(columns = {0:'p'}
              ).assign(sig = lambda df: df['p'].map(lambda p: ''    if p > 0.05  else
                                                              '*'   if p > 0.01  else
                                                              '**'  if p > 0.001 else
                                                              '***')) 

dunn_p_pair_table(difficulty_content_dunn_p).round(4)

> *Wniosek:* \
> Treść typu quiz różni się poziomem trudności ( $med = 3$ ; $IQR = 0$ ) od treści typu artykuł ( $med = 1$ ; $IQR = 1$ ) ( $p < 0.001$ ).
>
> Treść typu quiz różni się poziomem trudności od treści typu interaktywnego ( $med = 2$ ; $IQR = 2$ ) ( $p < 0.001$ ).
>
> Treść typu quiz różni się poziomem trudności od treści typu wideo ( $med = 1$ ; $IQR = 2$ ) ( $p < 0.001$ ).
>
> Dla pozostałych par typów treści na poziomie istotności 0.05 nie można stwierdzić istotnej różnicy w poziomie trudności.

Uwaga! W powyższym teście kategoria quiz nie spełniała wymaganej liczebności dla testu Kruskala-Wallisa (min. 5 obserwacji w każdej grupie zmiennej), więc nie jest to w pełni poprawnie wykonany test (ale sprawdzanie założeń tych testów już pominiemy).

# 3. Testy istotności różnic dla danych zależnych

## 3.1. 2 grupy

### Parametryczne
- *Zmienna ilościowa o rozkładzie normalnym w parach od tych samych obiektów*

    Test Studenta dla 2 prób zależnych

$H_0: \mu_1 = \mu_2$ \
$H_1: \mu_1 \neq \mu_2$  

Założenia:

- Badana zmienna ma w grupach rozkład normalny.

#### Przykład:

zmienna - pomiar stresu \
grupy - czas (przed/po interakcji)

Hipoteza badawcza: Pomiary stresu przed i po interakcji użytkownika z treścią różnią się od siebie.

$H_0:$ Średnia różnica między pomiarami stresu przed i po interakcji użytkownika z treścią wynosi 0. \
$H_1:$ Średnia różnica między pomiarami stresu przed i po interakcji użytkownika z treścią jest różna od 0.

**Rozkład zmiennej w grupach:**

In [ ]:
data[['stress_before', 'stress_after']].hist(bins = np.linspace(15, 35, 11),  grid = False)
data[['stress_before', 'stress_after']].describe()

**Test wstępny** - normalność zmiennej - test Shapiro-Wilka:

In [ ]:
pg.normality(data['stress_before'] - data['stress_after'])

> *Wniosek:* \
> Różnica między pomiarami stresu przed interakcją użytkownika z treścią i po tej interakcji nie ma rozkładu normalnego ( $ W = 0.94$ ; $p < 0.001$ ).

Różnica między pomiarami stresu nie spełnia założenia normalności w teście Studenta, ale gdyby spełniała należałoby wykonać w sposób przedstawiony poniżej.

**Test docelowy** - test Studenta dla 2 prób zależnych:

In [ ]:
# scipy

st.ttest_rel(data['stress_before'], data['stress_after'])

In [ ]:
# pingouin

pg.ttest(data['stress_before'], data['stress_after'], paired = True)

> *Wniosek:* \
> Pomiary stresu przed interakcją użytkownika z treścią ( $m = 25.13$ ; $sd = 5.95$ ) i po tej interakcji ( $m = 22.02$ ; $sd = 5.93$ ) różnią się od siebie ( $ T(299) = 45.62$ ; $p < 0.001$ ; $d = 0.52$ ).

### Nieparametryczne
- *Zmienna ilościowa o rozkładzie w parach od tych samych obiektów różnym od normalnego*
- *Zmienna porządkowa w parach od tych samych obiektów*

    Test Wilcoxona

$H_0: F_1 = F_2 $ \
$H_1: F_1 \neq F_2$  

#### Przykład:

zmienna - pomiar stresu \
grupy - czas (przed/po interakcji)

Hipoteza badawcza: Pomiary stresu przed i po interakcji użytkownika z treścią różnią się od siebie.

$H_0:$ Mediana różnicy między pomiarami stresu przed i po interakcji użytkownika z treścią wynosi 0. \
$H_1:$ Mediana różnicy między pomiarami stresu przed i po interakcji użytkownika z treścią jest różna od 0.

**Test docelowy** - test Wilcoxona:

In [ ]:
# scipy

st.wilcoxon(data['stress_before'], data['stress_after'])      # jeśli zmienna porządkowa, to w postaci rangowej

In [ ]:
# pingouin

pg.wilcoxon(data['stress_before'], data['stress_after'])      # jeśli zmienna porządkowa, to w postaci rangowej

> *Wniosek:* \
> Pomiary stresu przed interakcją użytkownika z treścią ( $med = 25.00$ ; $IQR = 11.00$ ) i po tej interakcji ( $med = 22.18$ ; $IQR = 10.08$ ) różnią się od siebie ( $ W = 0$ ; $p < 0.001$ ; $r_{bc} = 1$ ; $CL = 0.64$ ). 

## 3.2. Więcej niż 2 grupy

### Parametryczne

**Test wstępny:** test Shapiro-Wilka &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; (pg.normality(smf.ols(...)) \
**Test docelowy:** ANOVA dla powtarzanych pomiarów &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; (pg.rm_anova) \
**Testy post-hoc:** testy Wilcoxona dla par grup &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; (pg.pairwise_ttests(..., parametric = False, padjust = 'fdr_bh'))

### Nieparametryczne

**Test docelowy:** test Friedmana &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; (st.friedmanchisquare, pg.friedman) \
**Testy post-hoc:** testy Wilcoxona dla par grup &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; (sp.posthoc_conover_friedman(..., p_adjust = 'fdr_bh'), pg.pairwise_ttests(..., parametric = False, padjust = 'fdr_bh'))

# 4. Testy siły związku

## Parametryczne

- *Dwie zmienne ilościowe o rozkładzie normalnym*

    Test istotności współczynnika korelacji Pearsona

$H_0: \rho = 0 $ \
$H_1: \rho \neq 0$ 

Założenia:

- Łączny rozkład badanych zmiennych jest normalny.

#### Przykład:

zmienna 1 - wskaźnik uwagi \
zmienna 2 - wskaźnik zmienności nastroju

Hipoteza badawcza: Między wskaźnikami uwagi i zmienności nastroju istnieje związek.

$H_0:$ Korelacja pomiędzy wskaźnikami uwagi i zmienności nastroju wynosi 0. \
$H_1:$ Korelacja pomiędzy wskaźnikami uwagi i zmienności nastroju jest różna od 0.

**Rozkład zmiennych:**

In [ ]:
data.plot(x = 'attention_score', y = 'mood_variability_score', kind = 'scatter')

**Test wstępny** - normalność zmiennych - test Henze-Zirklera:

Hipoteza badawcza: Wskaźniki uwagi i zmienności nastroju mają dwuwymiarowy rozkład normalny.

$H_0:$ Wskaźniki uwagi i zmienności nastroju mają dwuwymiarowy rozkład normalny. \
$H_1:$ Wskaźniki uwagi i zmienności nastroju nie mają dwuwymiarowego rozkładu normalnego.

In [ ]:
pg.multivariate_normality(data[['attention_score', 'mood_variability_score']])

> *Wniosek:* \
> Na poziomie istotności 0.05 nie mamy podstaw do odrzucenia hipotezy zerowej ( $HZ = 0.58$ ; $p = 0.63$ ), a więc przyjmiemy, że wskaźnik uwagi i wskaźnik zmienności nastroju w trakcie interakcji z treściami wystarczająco spełniają założenie normalności testu istotności współczynnika korelacji Pearsona.

**Test docelowy** - test istotności współczynnika korelacji Pearsona:

In [ ]:
# scipy

st.pearsonr(data['attention_score'], data['mood_variability_score'])

In [ ]:
# pingouin

pg.corr(data['attention_score'], data['mood_variability_score'])

> *Wniosek:* \
> Na poziomie istotności 0.05 nie można stwierdzić, czy między wskaźnikami uwagi i zmienności nastroju istnieje związek ( $ r(298) = 0.11$ ; $p = 0.05$ ).

## Nieparametryczne

- *Dwie zmienne nienominalne, z których co najmniej jedna nie ma rozkładu normalnego*

    Test istotności współczynnika korelacji Spearmana

$H_0: \rho_S = 0 $ \
$H_1: \rho_S \neq 0$ 

#### Przykład:

zmienna 1 - trudność treści \
zmienna 2 - wskaźnik informacji zwrotnej

Hipoteza badawcza: Między trudnością treści i wskaźnikiem informacji zwrotnej istnieje związek.

$H_0:$ Korelacja pomiędzy trudnością treści i wskaźnikiem informacji zwrotnej wynosi 0. \
$H_1:$ Korelacja pomiędzy trudnością treści i wskaźnikiem informacji zwrotnej jest różna od 0.

**Rozkład zmiennych:**

In [ ]:
# dopisanie do danych zmiennych porządkowych w postaci rangowej

data['difficulty_rank']      = data['difficulty'].factorize(sort = True)[0] + 1
data['feedback_rating_rank'] = data['feedback_rating'].factorize(sort = True)[0] + 1 

In [ ]:
# rozkład

data.groupby('difficulty')['feedback_rating'].value_counts(sort = False).plot(kind = 'bar')
data.groupby('difficulty_rank')['feedback_rating_rank'].describe()

**Test docelowy** - test istotności współczynnika korelacji Spearmana:

In [ ]:
# scipy

st.spearmanr(data['difficulty_rank'], data['feedback_rating'])                 # zmienne porządkowe w postaci rangowej

In [ ]:
# pingouin

pg.corr(data['difficulty_rank'], data['feedback_rating'], method = 'spearman') # zmienne porządkowe w postaci rangowej

> *Wniosek:* \
> Między trudnością treści i wskaźnikiem informacji zwrotnej istnieje słaby związek o zgodnym kierunku ( $r_S = 0.14$ ; $p = 0.01$ ) - im większa trudność treści, tym wyższy wskaźnik informacji zwrotnej.

- *Dwie zmienne nominalne*

    Test niezależności zmiennych chi^2

$H_0: F_{12} = F_1 \cdot F_2$ \
$H_1: F_{12} \neq F_1 \cdot F_2 $ 

#### Przykład:

zmienna 1 - płeć  \
zmienna 2 - zawód

Hipoteza badawcza: Między płcią i zawodem, istnieje związek.

$H_0:$ Płeć i zawód są od siebie niezależne. \
$H_1:$ Płeć i zawód są od siebie zależne.

**Rozkład zmiennych:**

In [ ]:
users_data.groupby('gender')['occupation'].value_counts(sort = False).plot(kind = 'bar')
gender_occ_ctable = pd.crosstab(users_data['gender'], users_data['occupation'])
gender_occ_ctable

**Test docelowy** - test niezależności zmiennych $\chi^2$:

In [ ]:
# scipy

print(st.chi2_contingency(gender_occ_ctable, correction = False)[0:3])    # 'chi2', 'pval', 'dof'
print(st.contingency.association(gender_occ_ctable))                      # 'cramer'

In [ ]:
# pingouin

GO_E, GO_O, GO_test = pg.chi2_independence(users_data, x = 'gender', y = 'occupation', correction = False)
GO_test.loc[[0]]

In [ ]:
# zależność

GO_O - GO_E

> *Wniosek:* \
> Na poziomie istotności 0.05 nie można stwierdzić, czy między płcią i zawodem, istnieje związek ( $\chi^2(4) = 3.43$ ; $p = 0.48$ ; $V = 0.26$ ).

Uwaga! W powyższym teście dwa pola tabeli krzyżowej nie spełniały wymaganej liczebności dla testu $\chi^2$ (min. 5 obserwacji w każdym polu), więc test ten nie jest dobrze dobrany do problemu - należałoby wykonać dokładny test Fishera (patrz statistical_tests_tree.png), którego w Pythonie znajduje się implementacja jedynie dla tabel wymiaru 2x2.

---

Współczynniki korelacji stanowią jednocześnie miarę wielkości efektu.

Przykładowa interpretacja współczynnika korelacji Pearsona i Spearmana:

- $|r| < 0.4$ &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;         - związek słaby,
- $0.4 \leqslant |r| < 0.7$                                                        - związek umiarkowany,
- $0.7 \leqslant |r| < 0.9$                                                        - związek silny,
- $0.9 \leqslant |r|$ &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; - związek bardzo silny.

W przypadku współczynnika korelacji $V$ Craméra te same wartości wiążą się z większą siłą związku niż w przypadku współczynników korelacji Pearsona i Spearmana - im większa tabela krzyżowa, tym z wyższą.